# Query Rewrite and Web Search Context for LLMs

- This code snippet demonstrates how to rewrite a query, perform a web search, and prepare the results for use with a language model (LLM). It includes functions to rewrite the query using a language model, perform a web search using the rewritten query, and format the search results for the LLM.





In [1]:
import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import load_prompt
import sys
import logging
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search
from IPython.display import Markdown, display
from datetime import datetime
import time
import os
import pytz
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [2]:
# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
        
        prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
        QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
                  user_query=query)},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )

        print("Rewritten query for search:", response.choices[0].message.content.strip())
        
        return json.loads(response.choices[0].message.content.strip())


In [3]:

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_web_search(RESULTS_COUNT, input, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()

    print(f"############## Web Search Mode: {web_search_mode}")
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # 웹 검색 실행 (비동기)
    print("Search rewrited search queries...")   
    results = await web_search(query_rewrite, RESULTS_COUNT, web_search_mode=web_search_mode, product_name=product_name)

    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")


    # Get the current working directory for this notebook
    current_dir = os.getcwd()
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=results if results else '검색 결과 없음',
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
            
    # 검색 결과가 리스트(정상)인 경우 LLM 호출, 아니면 안내 메시지 출력
    
    print("Generate response...")   
    
    if web_search_mode == "google" and isinstance(results, list) and results:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=answer_messages,
            top_p=0.9,
            max_tokens=1500
        )
        display(Markdown(response.choices[0].message.content))
    elif web_search_mode == "bing" and isinstance(results, list) and results:
        for result in results:
            if isinstance(result, str):
                display(Markdown(result))
            else:
                display(Markdown(str(result)))

    else:
        display(Markdown("검색 결과가 없습니다. 다른 질문을 해주세요."))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [4]:
RESULTS_COUNT = 5

inputs = [
    "엄빠폰 느낌좋은걸로 추구미 비교 추천",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

web_search_mode = "bing"

for input in inputs:
    await process_web_search(RESULTS_COUNT, input, web_search_mode, product_name="삼성전자")  # product_name은 필요에 따라 변경 가능

Original Input: 엄빠폰 느낌좋은걸로 추구미 비교 추천
############## Web Search Mode: bing


Rewritten query for search: {
"search_query": "엄빠폰 느낌 좋은 추구미 비교 추천",
"llm_query": "엄빠폰의 느낌이 좋은 추구미 제품들을 비교하여 추천해 주세요. 각 제품의 주요 특징과 장단점도 함께 설명해 주시면 감사하겠습니다."
}
Web Search Query: 엄빠폰 느낌 좋은 추구미 비교 추천
LLM Query: 엄빠폰의 느낌이 좋은 추구미 제품들을 비교하여 추천해 주세요. 각 제품의 주요 특징과 장단점도 함께 설명해 주시면 감사하겠습니다.
Search rewrited search queries...
Generate response...


2025년 기준 엄빠폰으로 적합한 스마트폰 제품들을 비교하여 추천드립니다. 각 제품의 기본적인 특징, 장단점을 아래에서 확인하세요. 사용 목적이나 예산에 따라 선택할 수 있도록 다양한 옵션을 포함했습니다.

## 1. 삼성 갤럭시 A15
- **가격**: 약 25만 원
- **주요 특징**:
  - 화면: 6.5인치 HD 디스플레이
  - 배터리: 5000mAh
  - 사양: 안정적인 삼성 UI, 3년 보안 업데이트
- **장점**:
  - 삼성의 A/S망과 안정적인 고객 지원
  - 기본적인 기능만 필요한 사용자에게 적합
- **단점**:
  - 게임이나 고사양 앱 구동 시 성능 부족할 수 있음【3:1†source】【5:1†source】.

## 2. 삼성 갤럭시 점프3 (Galaxy Jump3)
- **가격**: 약 20~30만 원
- **주요 특징**:
  - 대화면 6.6인치, 5000mAh 배터리
  - 쉬운 모드 탑재로 사용 편리
- **장점**:
  - 삼성 헬스와 다양한 기능 지원
  - 부모님들이 쉽게 접근 가능한 UI
- **단점**:
  - 성능이 보급형 모델이므로 고사양 작업엔 한계가 있음【5:2†source】.

## 3. LG Q52/Q61
- **가격**: 약 15~25만 원
- **주요 특징**:
  - 6.5인치 디스플레이, 물리 버튼
  - 내구성이 뛰어난 디자인
- **장점**:
  - 쉽게 사용 가능한 물리 버튼으로 통화나 카메라 실행이 편리
  - 많은 사용자들에게 선택받은 효도폰
- **단점**:
  - 구형 모델이라 최신 소프트웨어 지원이 부족할 수 있음【5:2†source】.

## 4. 샤오미 레드미 노트 13 (Redmi Note 13)
- **가격**: 약 20만 원 초반
- **주요 특징**:
  - 6.67인치 AMOLED 디스플레이
  - 5000mAh 배터리
- **장점**:
  - 가격 대비 훌륭한 가성비
  - 화면이 커서 동영상 시청에 최적
- **단점**:
  - 중국 브랜드로, 삼성이나 LG에 비해 A/S가 불편할 수 있음【5:2†source】.

## 5. 와이즈폰2
- **가격**: 약 10만 원 전후
- **주요 특징**:
  - 진짜 노인 전용 기능: 큰 글씨, 물리버튼, 음성 읽기
  - SOS 버튼 등의 비상 기능
- **장점**:
  - 복잡한 기능 없이 편리하게 사용할 수 있도록 설계됨
  - 저렴한 가격으로 부모님께 선물하기에 적합
- **단점**:
  - 기본적인 기능 외에는 아쉬울 수 있음【5:2†source】.

이 목록의 각 스마트폰은 부모님이나 노인 사용자에게 적합한 다양한 기능과 가격대를 제공하므로 사용 용도와 예산을 고려하여 선택하시면 좋습니다. 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다.

- [2025년 부모님께 딱! 중저가 효도폰 추천 리스트](https://www.lifeinforwire.com/2025/04/parents-day-recommended-filial-piety-phone.html)【3:1†source】.
- [2025년 최신 노인용 효도폰 추천 TOP 5](https://goodsen2000.tistory.com/entry/2025%EB%85%84-%EC%B5%9C%EC%8B%A0-%EB%85%B8%EC%9D%B8%EC%9A%A9-%ED%9A%A8%EB%8F%84%ED%8F%B0-%EC%B6%94%EC%B2%9C-TOP-5-%E2%80%93-%EB%B6%80%EB%AA%A8%EB%8B%98%EA%BB%98-%EB%93%9C%EB%A6%AC%EB%A9%B4-%EC%A0%95%EB%A7%90-%EC%A2%8B%EC%95%84%ED%95%98%EC%8B%A4-%EA%B8%B0%EB%8A%A5-%EC%A4%91%EC%8B%AC%EC%9C%BC%EB%A1%9C)【5:2†source】.

elapsed time: 42.00 seconds


